In [1]:
pip install composable


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install composable composable-extras composable-records


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement composable-extras (from versions: none)
ERROR: No matching distribution found for composable-extras


In [3]:
import composable
print(dir(composable))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'operator', 'pipeable']


In [4]:
from composable import pipeable
from composable.strict import map, filter




import pandas as pd
import glob

In [6]:
from composable_utility import get, with_open, identity
from composable_object import obj, attr
from composable_origami import fold

In [2]:
pip install --upgrade --force-reinstall composable


  Using cached composable-0.5.4-py3-none-any.whl.metadata (696 bytes)
  Using cached python_forge-18.6.0-py35-none-any.whl.metadata (6.6 kB)
  Using cached toolz-0.11.2-py3-none-any.whl.metadata (5.1 kB)
Using cached composable-0.5.4-py3-none-any.whl (8.5 kB)
Using cached python_forge-18.6.0-py35-none-any.whl (31 kB)
Using cached toolz-0.11.2-py3-none-any.whl (55 kB)
  Attempting uninstall: python-forge
    Found existing installation: python-forge 18.6.0
    Uninstalling python-forge-18.6.0:
      Successfully uninstalled python-forge-18.6.0
  Attempting uninstall: toolz
    Found existing installation: toolz 0.11.2
    Uninstalling toolz-0.11.2:
      Successfully uninstalled toolz-0.11.2
  Attempting uninstall: composable
    Found existing installation: composable 0.5.4
    Uninstalling composable-0.5.4:
      Successfully uninstalled composable-0.5.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import composable
help(composable)


Help on package composable:

NAME
    composable

PACKAGE CONTENTS
    dict
    glob
    operator
    pipeable
    sequence
    strict
    string
    tuple

VERSION
    0.1.0

FILE
    c:\users\at3571rj\appdata\local\anaconda3\envs\polars\lib\site-packages\composable\__init__.py




In [3]:
pip install composable


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade composable


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install toolz cytoolz more-itertools


Note: you may need to restart the kernel to use updated packages.


In [1]:
import composable
print(dir(composable))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'operator', 'pipeable']


In [7]:
# Define paths for attendance files
paths = glob.glob("./attendance_example_fixed_width/*/Attendance Quiz - User Attempts.txt", recursive=True)
readme_path = "./attendance_example_fixed_width/README.txt"

In [8]:
# Helper functions for extracting information from file paths
get_file = lambda path: path.split('/')[-1]
get_folder = lambda path: '/'.join(path.split('/')[:-1]) + '/'
get_course_section = lambda path: path.split('/')[-2]  # Extract course and section from folder name

In [9]:
# Define fixed-width column sizes based on README
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]


In [10]:
# Read and process attendance files
paths_and_lines = [
    {
        'file': get_file(path),
        'folder': get_folder(path),
        'course_section': get_course_section(path),
        'lines': with_open(lambda f: f.readlines(), path)
    }
    for path in paths
]

In [11]:
# Parse fixed-width format
parse_fixed_width = pipeable(lambda line: [line[sum(column_widths[:i]):sum(column_widths[:i+1])].strip() for i in range(len(columns))])

# Extract data
header_and_body = [
    record | {
        'body': [parse_fixed_width(line) + [record['course_section']] for line in record['lines'][1:]],
    }
    for record in paths_and_lines
]

In [12]:
# Convert to DataFrame
df = pd.DataFrame(sum([rec['body'] for rec in header_and_body], []), columns=columns + ["CourseSection"])

In [13]:
# Convert AttemptNum to integer
df["AttemptNum"] = df["AttemptNum"].astype(int)

In [14]:
# Keep only max attempt per student
df = df.sort_values(by=["OrgDefinedID", "AttemptNum"], ascending=[True, False])
df = df.drop_duplicates(subset=["OrgDefinedID"], keep="first")

In [15]:
# Drop meaningless columns for attendance
df = df.drop(columns=["Score", "OutOf", "Percent"])


In [16]:
df = df.with_columns(
    pl.when(pl.col("AttemptStart").str.contains(" "))  # Check if it has both Date & Time
    .then(pl.col("AttemptStart").str.split(" "))  # Split into list
    .otherwise(pl.lit(["", ""]))  # If missing, replace with empty list
    .alias("DateTime")
)

df = df.with_columns(
    pl.col("DateTime").list.get(0).fill_null("").alias("Date"),  # Extract Date
    pl.when(pl.col("DateTime").list.len() > 1)  # Check if Time exists
    .then(pl.col("DateTime").list.get(1))
    .otherwise("")
    .alias("Time")
)

df = df.drop(["AttemptStart", "DateTime"])  # Remove old columns


AttributeError: 'DataFrame' object has no attribute 'with_columns'

In [22]:
# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = df[final_columns]

KeyError: "['Date', 'Time'] not in index"

In [ ]:
# Save to CSV
output_path = "./combined_attendance.csv"
df.to_csv(output_path, index=False)

print(f"Combined attendance file saved at {output_path}")


In [17]:
!pip install chardet


  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)


In [6]:
import os

# Define the base directory
base_path = os.path.abspath(".")  # Current working directory

print(f"🔍 Checking files and folders inside: {base_path}")

# Walk through all directories and files
for root, dirs, files in os.walk(base_path):
    level = root.replace(base_path, "").count(os.sep)
    indent = " " * (level * 4)
    print(f"{indent}📂 {os.path.basename(root)}/")
    
    sub_indent = " " * ((level + 1) * 4)
    for file in files:
        print(f"{sub_indent}📄 {file}")

print("\n✅ Directory structure displayed.")


🔍 Checking files and folders inside: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width
📂 attendance_example_fixed_width/
    📄 .DS_Store
    📄 combined_attendance.csv
    📄 README.txt
    📄 Untitled.ipynb
    📂 .ipynb_checkpoints/
        📄 combined_attendance-checkpoint.csv
        📄 README-checkpoint.txt
        📄 Untitled-checkpoint.ipynb
    📂 dsci494s7/
        📄 .DS_Store
        📄 Attendance Quiz - User Attempts.txt
        📄 Practice Quiz - Module 1 - User Attempts.txt
        📄 Practice Quiz - Module 2 - User Attempts.txt
        📄 Practice Quiz - Module 3 - User Attempts.txt
        📄 Practice Quiz - Module 4 - User Attempts.txt
        📂 .ipynb_checkpoints/
            📄 Attendance Quiz - User Attempts-checkpoint.txt
            📄 Practice Quiz - Module 1 - User Attempts-checkpoint.txt
    📂 stat180s18/
        📄 .DS_Store
        📄 Attendance Quiz - User Attempts.txt
    📂 stat491s1/
        📄 .DS_Store
        📄 Attendance Qu

In [5]:
!pip install composable


In [8]:
import composable
print(dir(composable))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'operator', 'pipeable', 'strict']


In [9]:

from composable import pipeable
from composable.strict import map
import glob
from composable_glob import glob
from composable_utility import with_open
import composable_records as rec
import composable_tuples as tup
import polars as pl
import os
import re


ModuleNotFoundError: No module named 'composable_glob'

In [18]:
import polars as pl
import glob
import os
import time

# Define base path for attendance files
base_path = os.path.abspath("C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width")
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")
paths = glob.glob(pattern, recursive=True)

# Debug: Print absolute path and found files
print("Searching in directory:", base_path)
if not os.path.exists(base_path):
    print("Error: Directory does not exist! Check if 'attendance_example_fixed_width' is in the correct location.")
    exit()

print("Files found:", paths)
if not paths:
    print("No files found! Ensure the correct file structure and check filenames.")
    print("Existing directories:", os.listdir(base_path))

# Helper functions for extracting metadata
def get_file(path):
    return os.path.basename(path)

def get_folder(path):
    return os.path.dirname(path)

def get_course_section(path):
    return os.path.basename(os.path.dirname(path))  # Extract course section from folder name

# Define fixed-width column sizes and headers
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]

# Function to parse fixed-width formatted line
def parse_fixed_width(line):
    parsed = []
    start = 0
    for width in column_widths:
        parsed.append(line[start:start + width].strip())
        start += width
    return parsed

# Read and process all attendance files from all subfolders
data = []
paths_and_lines = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        print(f"Reading file: {path}, Total lines: {len(lines)}")  # Debugging
        if len(lines) <= 1:
            print(f"Skipping empty file: {path}")
            continue
        
        # Skip header if present
        if "OrgDefinedID" in lines[0]:
            lines = lines[1:]
        
        for line in lines:
            parsed_line = parse_fixed_width(line) + [course_section]
            if len(parsed_line) == len(columns) + 1:  # Ensure proper parsing
                data.append(parsed_line)
    
    # Store metadata for each file
    paths_and_lines.append({
        'file': get_file(path),
        'folder': get_folder(path),
        'course_section': course_section,
        'total_lines': len(lines),
    })

# Debug: Print extracted metadata per file
print("✅ File metadata:", paths_and_lines)

# Debug: Print sample parsed data
if data:
    print("✅ Sample parsed data:", data[:5])
else:
    print("❌ No data extracted. Check file content and parsing function.")
    exit()

# Convert to Polars DataFrame
df = pl.DataFrame(data, schema=columns + ["CourseSection"], orient="row")

# Convert AttemptNum to integer
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))

# Group by student and compute total attendance across all subfolders
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),  # Sum all attempts for total attendance
    pl.col("AttemptStart").first()
)

# Ensure AttemptStart is a string and fill null values
df = df.with_columns(
    pl.col("AttemptStart").cast(pl.Utf8).fill_null("")
)

# Split Attempt_Start into Date and Time
df = df.with_columns(
    df["AttemptStart"].str.split(" ").alias("DateTime")
)
df = df.with_columns(
    df["DateTime"].list.get(0).fill_null("").alias("Date"),
    df["DateTime"].list.get(1).fill_null("").alias("Time")
)
df = df.drop(["AttemptStart", "DateTime"])

# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "TotalAttendance", "Date", "Time", "CourseSection"]
df = df.select(final_columns)

# Ensure the CSV file is not in use
output_path = os.path.join(base_path, "combined_attendance.csv")
retry_count = 5
while os.path.exists(output_path) and retry_count > 0:
    try:
        os.rename(output_path, output_path + ".tmp")  # Try renaming to check if file is open
        os.rename(output_path + ".tmp", output_path)  # Rename it back
        break
    except OSError:
        print("Error: The CSV file is currently open. Please close it and try again. Retrying...")
        time.sleep(2)
        retry_count -= 1

if retry_count == 0:
    print("Error: Could not save the file because it is in use. Please close it and try again.")
    exit()

# Save to CSV
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


Searching in directory: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width
Files found: ['C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']
Reading file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt, Total lines: 21
Reading file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat180s18\Attendance Quiz - User Attempts.txt, Total lines: 1
Skipping empty file: C:\Users\at3571rj\OneDri

In [19]:
import polars as pl
import glob
import os

# Define base path for attendance files
base_path = os.path.abspath("C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width")
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")
paths = glob.glob(pattern, recursive=True)

# Debug: Check for files
if not paths:
    print("No files found! Ensure the correct file structure and check filenames.")
    exit()

# Define fixed-width column sizes and headers
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]

# Function to parse fixed-width formatted line
def parse_fixed_width(line):
    parsed = []
    start = 0
    for width in column_widths:
        parsed.append(line[start:start + width].strip())
        start += width
    return parsed

# Read and process all attendance files from all subfolders
data = []
for path in paths:
    course_section = os.path.basename(os.path.dirname(path))  # Extract course section from folder name
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if len(lines) <= 1:
            print(f"Skipping empty file or header-only file: {path}")
            continue

        # Ensure the first line is a header and remove it
        if "OrgDefinedID" in lines[0]:
            lines = lines[1:]

        for line in lines:
            parsed_line = parse_fixed_width(line) + [course_section]
            if len(parsed_line) == len(columns) + 1:  # Ensure proper parsing
                data.append(parsed_line)

# Convert to Polars DataFrame
df = pl.DataFrame(data, schema=columns + ["CourseSection"], orient="row")

# Convert AttemptNum to integer
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))

# Group by student and compute total attendance across all subfolders
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),  # Sum all attempts for total attendance
    pl.col("AttemptStart").first()
)

# Ensure AttemptStart is a string and fill null values
df = df.with_columns(
    pl.col("AttemptStart").cast(pl.Utf8).fill_null("")
)

# Split AttemptStart into Date and Time
df = df.with_columns(
    df["AttemptStart"].str.split(" ").alias("DateTime")
)
df = df.with_columns(
    df["DateTime"].list.get(0).fill_null("").alias("Date"),
    df["DateTime"].list.get(1).fill_null("").alias("Time")
)
df = df.drop(["AttemptStart", "DateTime"])

# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "TotalAttendance", "Date", "Time", "CourseSection"]
df = df.select(final_columns)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Complete attendance file saved at {output_path}")


Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat180s18\Attendance Quiz - User Attempts.txt
Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat491s1\Attendance Quiz - User Attempts.txt
✅ Complete attendance file saved at C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\combined_attendance.csv


In [21]:
import polars as pl
import glob
import os
import time

# Define base path for attendance files
base_path = os.path.abspath("C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width")
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")
paths = glob.glob(pattern, recursive=True)

# Debug: Print absolute path and found files
print("Searching in directory:", base_path)
if not os.path.exists(base_path):
    print("Error: Directory does not exist! Check if 'attendance_example_fixed_width' is in the correct location.")
    exit()

print("Files found:", paths)
if not paths:
    print("No files found! Ensure the correct file structure and check filenames.")
    print("Existing directories:", os.listdir(base_path))

# Helper functions for extracting metadata
def get_file(path):
    return os.path.basename(path)

def get_folder(path):
    return os.path.dirname(path)

def get_course_section(path):
    return os.path.basename(os.path.dirname(path))  # Extract course section from folder name

# Define fixed-width column sizes and headers
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]

# Function to parse fixed-width formatted line
def parse_fixed_width(line):
    parsed = []
    start = 0
    for width in column_widths:
        parsed.append(line[start:start + width].strip())
        start += width
    return parsed

# Read and process all attendance files from all subfolders
data = []
paths_and_lines = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        print(f"Reading file: {path}, Total lines: {len(lines)}")  # Debugging
        if len(lines) <= 1:
            print(f"Skipping empty file: {path}")
            continue
        
        # Skip header if present
        if "OrgDefinedID" in lines[0]:
            lines = lines[1:]
        
        for line in lines:
            parsed_line = parse_fixed_width(line) + [course_section]
            if len(parsed_line) == len(columns) + 1:  # Ensure proper parsing
                data.append(parsed_line)
    
    # Store metadata for each file
    paths_and_lines.append({
        'file': get_file(path),
        'folder': get_folder(path),
        'course_section': course_section,
        'total_lines': len(lines),
    })

# Debug: Print extracted metadata per file
print("✅ File metadata:", paths_and_lines)

# Debug: Print sample parsed data
if data:
    print("✅ Sample parsed data:", data[:5])
else:
    print("❌ No data extracted. Check file content and parsing function.")
    exit()

# Convert to Polars DataFrame
df = pl.DataFrame(data, schema=columns + ["CourseSection"], orient="row")

# Convert AttemptNum to integer
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))

# Group by student and compute total attendance across all subfolders
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),  # Sum all attempts for total attendance
    pl.col("AttemptStart").first()
)

# Ensure AttemptStart is a string and fill null values
df = df.with_columns(
    pl.col("AttemptStart").cast(pl.Utf8).fill_null("")
)

# Split Attempt_Start into Date and Time
df = df.with_columns(
    df["AttemptStart"].str.split(" ").alias("DateTime")
)
df = df.with_columns(
    df["DateTime"].list.get(0).fill_null("").alias("Date"),
    df["DateTime"].list.get(1).fill_null("").alias("Time")
)
df = df.drop(["AttemptStart", "DateTime"])

# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "TotalAttendance", "Date", "Time", "CourseSection"]
df = df.select(final_columns)

# Ensure the CSV file is not in use
output_path = os.path.join(base_path, "combined_attendance.csv")
retry_count = 5
while os.path.exists(output_path) and retry_count > 0:
    try:
        os.rename(output_path, output_path + ".tmp")  # Try renaming to check if file is open
        os.rename(output_path + ".tmp", output_path)  # Rename it back
        break
    except OSError:
        print("Error: The CSV file is currently open. Please close it and try again. Retrying...")
        time.sleep(2)
        retry_count -= 1

if retry_count == 0:
    print("Error: Could not save the file because it is in use. Please close it and try again.")
    exit()

# Save to CSV
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


Searching in directory: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width
Files found: ['C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:\\Users\\at3571rj\\OneDrive - MNSCU\\Desktop\\DSCI 330\\DSCI Project 1\\attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']
Reading file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt, Total lines: 21
Reading file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat180s18\Attendance Quiz - User Attempts.txt, Total lines: 1
Skipping empty file: C:\Users\at3571rj\OneDri

In [23]:
from composable_glob import glob
from composable_utility import with_open
import polars as pl
import os
import re

# Define the base path where attendance files are stored
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# 🔍 Get all attendance files in subfolders
paths = glob(pattern, recursive=True)

print("✅ Attendance Files Found:", paths)

# 📂 Helper function to extract metadata
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# 🔄 Unfolding file paths into records
paths_and_lines = [
    {'course_section': get_course_section(path), 'lines': with_open(lambda f: f.readlines(), path)}
    for path in paths
]

# 🏷️ Extract headers and bodies
header_and_body = [
    record | {
        'body': record['lines'][1:] if len(record['lines']) > 1 else []  # Extract data lines, ignoring headers
    }
    for record in paths_and_lines
]

# 🔄 Process and clean data
def process_line(line, course_section):
    """Splits each row properly while ensuring correct number of columns."""
    parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
    if len(parsed_line) >= 8:  # Adjust if more fields are expected
        return parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]  # Ensure correct length
    else:
        return None  # Ignore malformed rows

# 📝 Process all records into structured data
processed_records = [
    [process_line(line, record['course_section']) for line in record['body']]
    for record in header_and_body
]

# 📊 Flatten the list and remove None values
all_data = [row for sublist in processed_records for row in sublist if row]

# 🔹 Convert to a DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns)

# 📈 Summarizing attendance per student across all folders
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),
    pl.col("Date").first(),
    pl.col("Time").first()
)

# 📁 Save as CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']
✅ Combined attendance file saved at C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\combined_attendance.csv


C:\Users\at3571rj\AppData\Local\Temp\ipykernel_25592\3444368012.py:53: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(all_data, schema=columns)


In [26]:
from composable_glob import glob
from composable_utility import with_open
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("✅ Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r") as f:
        lines = f.readlines()
    
    print(f"\n📄 File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns)

# Debugging: Print out the first 5 rows
print("\n🔍 Sample of parsed data:")
print(df.head(5))

# Summarizing attendance per student
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),
    pl.col("Date").first(),
    pl.col("Time").first()
)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']

📄 File: C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt - Total Lines: 21
🔹 First 5 lines of C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt:
 ['14460432 au9747cp Jericho     Greer       1  1  1  2019-01-14 14:00:00 2019-01-14 14:06:00 100%14460432 au9747cp Jericho     Greer       2  1  1  2019-01-16 14:00:00 2019-01-16 14:08:00 1

C:\Users\at3571rj\AppData\Local\Temp\ipykernel_25592\1730032265.py:47: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(all_data, schema=columns)


In [27]:
from composable_glob import glob
from composable_utility import with_open
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("\u2705 Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r") as f:
        lines = f.readlines()
    
    print(f"\n\ud83d\udcdd File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns)

# Convert attempt number to integer
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))

# Ensure uniqueness by keeping the first occurrence of each unique user-course section pair
df = df.sort(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection", "AttemptNum", "Date", "Time"]).unique(
    subset=["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"], keep="first"
)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


C:\Users\at3571rj\AppData\Local\Temp\ipykernel_25592\3358691523.py:47: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(all_data, schema=columns)
Exception in callback BaseAsyncIOLoop._handle_events(2164, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(2164, 1)>
Traceback (most recent call last):
  File "C:\Users\at3571rj\AppData\Local\anaconda3\envs\polars\Lib\site-packages\jupyter_client\session.py", line 100, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 518-519: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\at3571rj\AppData\Local\anaconda3\envs\polars\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self

In [28]:
from composable_glob import glob
from composable_utility import with_open
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("\u2705 Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r") as f:
        lines = f.readlines()
    
    print(f"\n\ud83d\udcdd File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns)

# Convert attempt number to integer
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32))

# Ensure uniqueness by keeping the first occurrence of each unique user-course section pair
df = df.sort(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection", "AttemptNum", "Date", "Time"]).unique(
    subset=["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"], keep="first"
)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


C:\Users\at3571rj\AppData\Local\Temp\ipykernel_25592\3358691523.py:47: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(all_data, schema=columns)
Exception in callback BaseAsyncIOLoop._handle_events(2164, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(2164, 1)>
Traceback (most recent call last):
  File "C:\Users\at3571rj\AppData\Local\anaconda3\envs\polars\Lib\site-packages\jupyter_client\session.py", line 100, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 518-519: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\at3571rj\AppData\Local\anaconda3\envs\polars\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self

In [29]:
from composable_glob import glob
from composable_utility import with_open
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("✅ Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()
    
    print(f"\n📄 File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns, orient="row")  # Explicitly set orientation

# Ensure proper types
df = df.with_columns(
    pl.col("AttemptNum").cast(pl.Int32, strict=False),
    pl.col("Date").cast(pl.Utf8, strict=False),
    pl.col("Time").cast(pl.Utf8, strict=False)
)

# Remove duplicate entries, keeping the first occurrence
df = df.unique(subset=["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"])

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Combined attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']

📄 File: C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt - Total Lines: 21
🔹 First 5 lines of C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt:
 ['14460432 au9747cp Jericho     Greer       1  1  1  2019-01-14 14:00:00 2019-01-14 14:06:00 100%14460432 au9747cp Jericho     Greer       2  1  1  2019-01-16 14:00:00 2019-01-16 14:08:00 1

In [30]:
from composable_glob import glob
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("✅ Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()
    
    print(f"\n📄 File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns, orient="row")

# Ensure proper data types
df = df.with_columns(
    pl.col("AttemptNum").cast(pl.Int32, strict=False),
    pl.col("Date").cast(pl.Utf8, strict=False),
    pl.col("Time").cast(pl.Utf8, strict=False)
)

# Aggregate all records for each unique student
df_grouped = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName"]).agg([
    pl.col("AttemptNum").list().alias("All_Attempts"),
    pl.col("Date").list().alias("All_Dates"),
    pl.col("Time").list().alias("All_Times"),
    pl.col("CourseSection").list().alias("All_CourseSections")
])

# Save to CSV
output_path = os.path.join(base_path, "merged_attendance.csv")
df_grouped.write_csv(output_path)

print(f"✅ Merged attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']

📄 File: C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt - Total Lines: 21
🔹 First 5 lines of C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt:
 ['14460432 au9747cp Jericho     Greer       1  1  1  2019-01-14 14:00:00 2019-01-14 14:06:00 100%14460432 au9747cp Jericho     Greer       2  1  1  2019-01-16 14:00:00 2019-01-16 14:08:00 1

TypeError: 'ExprListNameSpace' object is not callable

In [31]:
from composable_glob import glob
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("✅ Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()
    
    print(f"\n📄 File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns, orient="row")

# Ensure proper data types
df = df.with_columns(
    pl.col("AttemptNum").cast(pl.Int32, strict=False),
    pl.col("Date").cast(pl.Utf8, strict=False),
    pl.col("Time").cast(pl.Utf8, strict=False)
)

# Aggregate all records for each unique student
df_grouped = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName"]).agg(
    pl.col("AttemptNum").alias("All_Attempts").list(),
    pl.col("Date").alias("All_Dates").list(),
    pl.col("Time").alias("All_Times").list(),
    pl.col("CourseSection").alias("All_CourseSections").list()
)

# Save to CSV
output_path = os.path.join(base_path, "merged_attendance.csv")
df_grouped.write_csv(output_path)

print(f"✅ Merged attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']

📄 File: C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt - Total Lines: 21
🔹 First 5 lines of C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt:
 ['14460432 au9747cp Jericho     Greer       1  1  1  2019-01-14 14:00:00 2019-01-14 14:06:00 100%14460432 au9747cp Jericho     Greer       2  1  1  2019-01-16 14:00:00 2019-01-16 14:08:00 1

TypeError: 'ExprListNameSpace' object is not callable

In [32]:
from composable_glob import glob
import polars as pl
import os
import re

# Define base path
base_path = "C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width"
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")

# Get all attendance files
paths = glob(pattern, recursive=True)
print("✅ Attendance Files Found:", paths)

# Helper function to extract the course section (subfolder name)
get_course_section = lambda path: os.path.basename(os.path.dirname(path))

# Process each file
all_data = []
for path in paths:
    course_section = get_course_section(path)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        lines = f.readlines()
    
    print(f"\n📄 File: {path} - Total Lines: {len(lines)}")
    if len(lines) == 0:
        print(f"⚠️ Warning: Empty file {path}, but it will still be included.")
        continue

    # Ignore the first header row
    lines = lines[1:] if "OrgDefinedID" in lines[0] else lines

    # Debugging: Print the first 5 lines
    print(f"🔹 First 5 lines of {path}:\n", lines[:5])

    # Process each line
    for line in lines:
        parsed_line = re.split(r'\s+', line.strip())  # Split on spaces
        if len(parsed_line) >= 8:  # Adjust to match column count
            record = parsed_line[:5] + [parsed_line[5], parsed_line[6], course_section]
            all_data.append(record)
        else:
            print(f"⚠️ Skipping malformed row: {parsed_line} (Expected at least 8 columns)")

# Convert to DataFrame
columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum", "Date", "Time", "CourseSection"]
df = pl.DataFrame(all_data, schema=columns, orient="row")

# Ensure proper data types
df = df.with_columns(
    pl.col("AttemptNum").cast(pl.Int32, strict=False),
    pl.col("Date").cast(pl.Utf8, strict=False),
    pl.col("Time").cast(pl.Utf8, strict=False)
)

# Aggregate all records for each unique student
df_grouped = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName"]).agg(
    pl.col("AttemptNum").alias("All_Attempts").list(),
    pl.col("Date").alias("All_Dates").list(),
    pl.col("Time").alias("All_Times").list(),
    pl.col("CourseSection").alias("All_CourseSections").list()
)

# Save to CSV
output_path = os.path.join(base_path, "merged_attendance.csv")
df_grouped.write_csv(output_path)

print(f"✅ Merged attendance file saved at {output_path}")


✅ Attendance Files Found: ['C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\dsci494s7\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat180s18\\Attendance Quiz - User Attempts.txt', 'C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\\stat491s1\\Attendance Quiz - User Attempts.txt']

📄 File: C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt - Total Lines: 21
🔹 First 5 lines of C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width\dsci494s7\Attendance Quiz - User Attempts.txt:
 ['14460432 au9747cp Jericho     Greer       1  1  1  2019-01-14 14:00:00 2019-01-14 14:06:00 100%14460432 au9747cp Jericho     Greer       2  1  1  2019-01-16 14:00:00 2019-01-16 14:08:00 1

TypeError: 'ExprListNameSpace' object is not callable

In [33]:
import polars as pl
import glob
import os

# Define base path for attendance files
base_path = os.path.abspath("C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width")
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")
paths = glob.glob(pattern, recursive=True)

# Debug: Check for files
if not paths:
    print("No files found! Ensure the correct file structure and check filenames.")
    exit()

# Define fixed-width column sizes and headers
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]

# Function to parse fixed-width formatted line
def parse_fixed_width(line):
    parsed = []
    start = 0
    for width in column_widths:
        parsed.append(line[start:start + width].strip() or "")  # Ensure missing values are kept as empty strings
        start += width
    return parsed

# Read and process all attendance files from all subfolders
data = []
for path in paths:
    course_section = os.path.basename(os.path.dirname(path))  # Extract course section from folder name
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if len(lines) <= 1:
            print(f"Skipping empty file or header-only file: {path}")
            continue

        # Ensure the first line is a header and remove it
        if "OrgDefinedID" in lines[0]:
            lines = lines[1:]

        for line in lines:
            parsed_line = parse_fixed_width(line) + [course_section]
            data.append(parsed_line)

# Convert to Polars DataFrame
df = pl.DataFrame(data, schema=columns + ["CourseSection"], orient="row")

# Preserve all information by treating all columns as strings first
df = df.with_columns([pl.col(col).cast(pl.Utf8).fill_null("") for col in columns])

# Convert AttemptNum to integer (if possible) while preserving missing values
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32, strict=False).fill_null(0))

# Group by student and compute total attendance across all subfolders
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),  # Sum all attempts for total attendance
    pl.col("AttemptStart").first().fill_null("")
)

# Split AttemptStart into Date and Time (without skipping)
df = df.with_columns(
    df["AttemptStart"].str.split(" ").alias("DateTime")
)
df = df.with_columns(
    df["DateTime"].list.get(0).fill_null("").alias("Date"),
    df["DateTime"].list.get(1).fill_null("").alias("Time")
)

# Drop old AttemptStart and temporary DateTime column
df = df.drop(["AttemptStart", "DateTime"])

# Ensure no rows are dropped due to missing data
df = df.with_columns([pl.col(col).fill_null("") for col in df.columns])

# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "TotalAttendance", "Date", "Time", "CourseSection"]
df = df.select(final_columns)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Complete attendance file saved at {output_path}")


Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat180s18\Attendance Quiz - User Attempts.txt
Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat491s1\Attendance Quiz - User Attempts.txt
✅ Complete attendance file saved at C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\combined_attendance.csv


In [34]:
import polars as pl
import glob
import os

# Define base path for attendance files
base_path = os.path.abspath("C:/Users/at3571rj/OneDrive - MNSCU/Desktop/DSCI 330/DSCI Project 1/attendance_example_fixed_width")
pattern = os.path.join(base_path, "*/Attendance Quiz - User Attempts.txt")
paths = glob.glob(pattern, recursive=True)

# Debug: Check for files
if not paths:
    print("No files found! Ensure the correct file structure and check filenames.")
    exit()

# Define fixed-width column sizes and headers
column_widths = [9, 9, 12, 12, 3, 3, 3, 20, 20, 6]
columns = [
    "OrgDefinedID", "UserName", "FirstName", "LastName", "AttemptNum",
    "Score", "OutOf", "AttemptStart", "AttemptEnd", "Percent"
]

# Function to parse fixed-width formatted line
def parse_fixed_width(line):
    parsed = []
    start = 0
    for width in column_widths:
        parsed.append(line[start:start + width].strip() or "")  # Ensure missing values are kept as empty strings
        start += width
    return parsed

# Read and process all attendance files from all subfolders
data = []
for path in paths:
    course_section = os.path.basename(os.path.dirname(path))  # Extract course section from folder name
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if len(lines) <= 1:
            print(f"Skipping empty file or header-only file: {path}")
            continue

        # Ensure the first line is a header and remove it
        if "OrgDefinedID" in lines[0]:
            lines = lines[1:]

        for line in lines:
            parsed_line = parse_fixed_width(line) + [course_section]
            data.append(parsed_line)

# Convert to Polars DataFrame
df = pl.DataFrame(data, schema=columns + ["CourseSection"], orient="row")

# Preserve all information by treating all columns as strings first
df = df.with_columns([pl.col(col).cast(pl.Utf8).fill_null("") for col in columns])

# Convert AttemptNum to integer (if possible) while preserving missing values
df = df.with_columns(pl.col("AttemptNum").cast(pl.Int32, strict=False).fill_null(0))

# Ensure each student appears in **every course section they attended**
df = df.group_by(["OrgDefinedID", "UserName", "FirstName", "LastName", "CourseSection"]).agg(
    pl.col("AttemptNum").sum().alias("TotalAttendance"),  # Keep attendance per course section
    pl.col("AttemptStart").first().fill_null("")
)

# Split AttemptStart into Date and Time (without skipping)
df = df.with_columns(
    df["AttemptStart"].str.split(" ").alias("DateTime")
)
df = df.with_columns(
    df["DateTime"].list.get(0).fill_null("").alias("Date"),
    df["DateTime"].list.get(1).fill_null("").alias("Time")
)

# Drop old AttemptStart and temporary DateTime column
df = df.drop(["AttemptStart", "DateTime"])

# Ensure no rows are dropped due to missing data
df = df.with_columns([pl.col(col).fill_null("") for col in df.columns])

# Reorder columns
final_columns = ["OrgDefinedID", "UserName", "FirstName", "LastName", "TotalAttendance", "Date", "Time", "CourseSection"]
df = df.select(final_columns)

# Save to CSV
output_path = os.path.join(base_path, "combined_attendance.csv")
df.write_csv(output_path)

print(f"✅ Complete attendance file saved at {output_path}")


Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat180s18\Attendance Quiz - User Attempts.txt
Skipping empty file or header-only file: C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\stat491s1\Attendance Quiz - User Attempts.txt
✅ Complete attendance file saved at C:\Users\at3571rj\OneDrive - MNSCU\Desktop\DSCI 330\DSCI Project 1\attendance_example_fixed_width\combined_attendance.csv


In [10]:
from composable import pipeable
from composable.strict import map, filter
from composable_glob import glob
from composable_utility import with_open
import composable_records as rec
import composable_tuples as tup

In [11]:
from composable_utility import get, with_open, identity
from composable_object import obj, attr
from composable_origami import fold
import pandas as pd
from functools import reduce

In [13]:
# Define the glob pattern to match attendance files in subdirectories
paths = (
    "./attendance_example_fixed_width/*/*.txt"
    >> glob(recursive=True)
)

# Check if files are found
if not paths:
    print("No files found! Ensure the correct directory structure and filenames.")
else:
    print(f"Found {len(paths)} files.")

No files found! Ensure the correct directory structure and filenames.


In [14]:
import os

base_path = os.path.abspath("./attendance_example_fixed_width")
paths = (
    f"{base_path}/*/*.txt"
    >> glob(recursive=True)
)

print("Found files:", paths)


Found files: []
